# Robust mean-variance portfolio selection

### Import libraries

In [2]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#np.random.seed(1337)
#np.random.seed(178365)
np.random.seed(978351)

### Generate random data for 10 stocks

In [4]:
# Random data for 10 stocks
n = 10
Q = np.random.random((n,n))
Q = np.dot(Q,Q.T)/1000
# Q
mu = np.random.rand(n) / 100
# mu

### Equally-weighted portfolio

In [5]:
# Define initial portfolio ("equally weighted" or "1/n portfolio")
w0 = [1.0/n] * n
w0

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [6]:
# Sanity check
Sum_w = sum(w0)
# 1/n portfolio return
ret_init = np.dot(mu, w0)
# 1/n portfolio variance
var_init = np.dot(w0, np.dot(Q, w0))

In [7]:
print('For original (equally weighted) portfolio:')
print('  sum of asset weights = {0:0.2f}'.format(Sum_w))
print('  portfolio return  = {0:12.10f}'.format(ret_init))
print('  portfolio st. dev = {0:12.10f}'.format(np.sqrt(var_init)))

For original (equally weighted) portfolio:
  sum of asset weights = 1.00
  portfolio return  = 0.0038235552
  portfolio st. dev = 0.0515116026


In [8]:
# Required portfolio robustness
var_matr = np.diag(np.diag(Q))
# Target portfolio return estimation error is return estimation error of 1/n portfolio
rob_init = np.dot(w0, np.dot(var_matr, w0)) # return estimation error of initial portfolio
rob_bnd  = rob_init # target return estimation error

### Compute minimum variance portfolio

\begin{equation}
\begin{array}{rl}
\displaystyle \min_{w} & w^TQw \\
{\rm s.t.} & \sum_i w_i = 1\\
& w \geq 0
\end{array}
\end{equation}

In [9]:
w1 = cp.Variable(n)
prob1 = cp.Problem(cp.Minimize(cp.quad_form(w1, Q)),
                 [sum(w1) == 1,
                  w1 >= 0])
prob1.solve(solver=cp.CPLEX, verbose=True,cplex_params={"qpmethod": 6})

# Print results
print("\nSolution status: ", prob1.status)
print("Solution optimal value: ", prob1.value)
print("Solution w: ")
print(w1.value)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 24 02:07:59 PM: Your problem has 10 variables, 2 constraints, and 0 parameters.
(CVXPY) Jan 24 02:07:59 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 24 02:07:59 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 24 02:07:59 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 24 02:07:59 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 24 02:07:59 PM: Compiling problem (target solver=CPLEX).
(CV

In [10]:
# Check constraint
np.allclose(np.sum(w1.value),1)

True

In [11]:
w_minVar = w1.value
var_minVar = np.dot(w_minVar, np.dot(Q, w_minVar))
ret_minVar = np.dot(mu, w_minVar)
rob_minVar = np.dot(w_minVar, np.dot(var_matr, w_minVar))
print("Minimum variance portfolio:\n")
print("   Solution status =", prob1.status)
print("    Solution value =", prob1.value)
print("          Variance =", var_minVar)
print("   Expected return =", ret_minVar)
print("Standard deviation =", np.sqrt(var_minVar))

Minimum variance portfolio:

   Solution status = optimal
    Solution value = 0.001899092402601308
          Variance = 0.001899092402601308
   Expected return = 0.0032674778416423404
Standard deviation = 0.043578577335673864


### Compute robust mean-variance portfolio

\begin{equation}
  \begin{array}{lll}
    \min   & w^TQw       \\
    {\rm s.t.} & \begin{array}[t]{@{\hspace{0cm}}r@{\hspace{0.1cm}}c@{\hspace{0.1cm}}ll}
          \sum_{i=1}^n w_i & = & 1  \\
          \mu^T w & \geq & \varepsilon_{\rm ret} \\
          w^T \Theta w & \leq & \tilde{\varepsilon}_{\rm rob} \\
            w & \geq & 0
     \end{array}
  \end{array}
\end{equation}

In [12]:
# Target portfolio return is return of minimum variance portfolio
Portf_Retn = ret_minVar

In [13]:
Qq_rMV = var_matr
Qq_rMVs = np.sqrt(Qq_rMV)

In [14]:
w2 = cp.Variable(n)
prob2 = cp.Problem(cp.Minimize(cp.quad_form(w2, Q)),
                 [sum(w2) == 1,
                  mu.T@w2 >= Portf_Retn,
                  cp.quad_form(w2, Qq_rMV) <= rob_bnd,
                  w2 >= 0, w2 <= 1])
prob2.solve(solver=cp.CPLEX, verbose=True)

# Print results
print("\nSolution status: ", prob2.status)
print("Solution optimal value: ", prob2.value)
print("Solution w: ")
print(w2.value)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 24 02:08:12 PM: Your problem has 10 variables, 5 constraints, and 0 parameters.
(CVXPY) Jan 24 02:08:12 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 24 02:08:12 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 24 02:08:12 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 24 02:08:12 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 24 02:08:12 PM: Compiling problem (target solver=CPLEX).
(CV

In [15]:
w_rob1 = w2.value
var_rob1 = np.dot(w_rob1, np.dot(Q, w_rob1))
ret_rob1 = np.dot(mu, w_rob1)
print("Robust mean-varinace portfolio:\n")
print("   Solution status =", prob2.status)
print("    Solution value =", prob2.value)
print("          Variance =", var_rob1)
print("   Expected return =", ret_rob1)
print("Standard deviation =", np.sqrt(var_rob1))

Robust mean-varinace portfolio:

   Solution status = optimal
    Solution value = 0.002306815645407978
          Variance = 0.002306815645407978
   Expected return = 0.003380178892442723
Standard deviation = 0.04802932068443169


### Define and solve CPLEX robust mean-variance model directly without CVXPY

In [16]:
import cplex

In [17]:
cpx = cplex.Cplex()
cpx.objective.set_sense(cpx.objective.sense.minimize)

In [18]:
c  = [0.0] * n
lb = [0.0] * n
ub = [1.0] * n

In [19]:
A = []
for k in range(n):
    A.append([[0,1],[1.0,mu[k]]])
A

[[[0, 1], [1.0, 0.003576727911786397]],
 [[0, 1], [1.0, 0.00344522590632511]],
 [[0, 1], [1.0, 0.0007969492511468001]],
 [[0, 1], [1.0, 0.00018690160307030746]],
 [[0, 1], [1.0, 0.0035750515214236366]],
 [[0, 1], [1.0, 0.00893492393857598]],
 [[0, 1], [1.0, 0.005423138353068148]],
 [[0, 1], [1.0, 0.004447761567604534]],
 [[0, 1], [1.0, 0.007265204890217466]],
 [[0, 1], [1.0, 0.0005836669139738493]]]

In [20]:
var_names = ["w_%s" % i for i in range(1,n+1)]
var_names

['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 'w_7', 'w_8', 'w_9', 'w_10']

In [21]:
cpx.linear_constraints.add(rhs=[1.0,Portf_Retn], senses="EG")

range(0, 2)

In [22]:
cpx.variables.add(obj=c, lb=lb, ub=ub, columns=A, names=var_names)

range(0, 10)

In [23]:
Qmat = [[list(range(n)), list(2*Q[k,:])] for k in range(n)]

In [24]:
cpx.objective.set_quadratic(Qmat)

In [25]:
Qcon = cplex.SparseTriple(ind1=var_names, ind2=range(n), val=np.diag(var_matr))
cpx.quadratic_constraints.add(rhs=rob_bnd, quad_expr=Qcon, name="Qc")

0

In [26]:
cpx.parameters.threads.set(4)
print("Setting number of threads = ", 4)
cpx.parameters.timelimit.set(60)
print("Setting timelimit = ", 60)
cpx.parameters.barrier.qcpconvergetol.set(1e-12)
print("Setting Barrier algorithm convergence tolerance = ", 1e-12)

Setting number of threads =  4
Setting timelimit =  60
Setting Barrier algorithm convergence tolerance =  1e-12


In [27]:
# Disable CPLEX output to screen
#cpx.set_results_stream(None)
#cpx.set_warning_stream(None)
import time
start = time.time()
cpx.solve()
duration = time.time() - start

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
CPXPARAM_TimeLimit                               60
CPXPARAM_Barrier_QCPConvergeTol                  9.9999999999999998e-13
Tried aggregator 1 time.
Aggregator did 1 substitutions.
Reduced QCP has 35 rows, 33 columns, and 119 nonzeros.
Reduced QCP has 2 quadratic constraints.
Presolve time = 0.00 sec. (0.03 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 302
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.02 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 35
  Integer space required    = 84
  Total non-zeros in factor = 446
  Total FP ops to factor    = 7550
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0   1.8284271e+00  -1.0000000e+00  2.43

In [28]:
w_rMV = cpx.solution.get_values()
card_rMV = np.count_nonzero(w_rMV)
ret_rMV  = np.dot(mu, w_rMV)
var_rMV = np.dot(w_rMV, np.dot(Q, w_rMV))
rob_rMV = np.dot(w_rMV, np.dot(var_matr, w_rMV))

In [29]:
print('            Solution status =', cpx.solution.get_status())
print('     Solution status string =', cpx.solution.status[cpx.solution.get_status()])
print('              Solution time = {0:4.3f} seconds'.format(duration))
print('         Solution objective = {0:10.8f}'.format(cpx.solution.get_objective_value()))

            Solution status = 1
     Solution status string = optimal
              Solution time = 0.064 seconds
         Solution objective = 0.00230681


In [30]:
if(cpx.solution.get_status() == 1):
    print('      Portfolio cardinality = {0:d}'.format(card_rMV))
    print('         Solution objective = {0:10.8f}'.format(cpx.solution.get_objective_value()))
    print('     Portfolio rMV variance = {0:10.8f}'.format(var_rMV))
    print('      Sum of asset weights = {0:10.7f}'.format(np.sum(w_rMV)))
    print('\n       Portfolio rMV return = {0:9.5f}'.format(ret_rMV))
    print('    Portfolio minVar return = {0:9.5f}'.format(ret_minVar))
    print('      Portfolio init return = {0:9.5f}'.format(ret_init))   
    print('\n      Portfolio rMV st.dev. = {0:9.5f}'.format(np.sqrt(var_rMV)))
    print('   Portfolio minVar st.dev. = {0:9.5f}'.format(np.sqrt(var_minVar)))
    print('     Portfolio init st.dev. = {0:9.5f}'.format(np.sqrt(var_init)))
    print('\n   Portfolio rMV r.est.err. = {0:9.5f}'.format(np.sqrt(rob_rMV)))
    print(' Portfolio r.est.err. bound = {0:9.5f}'.format(np.sqrt(rob_bnd)))
    print('Portfolio minVar r.est.err. = {0:9.5f}'.format(np.sqrt(rob_minVar)))
    print('  Portfolio init r.est.err. = {0:9.5f}'.format(np.sqrt(rob_init)))
    print('\n   # of zero portf. weights = {0}'.format(np.sum((np.array(w_rMV) == 0))))
    print('  # of positive pf. weights = {0}'.format(np.sum((np.array(w_rMV) > 0))))

      Portfolio cardinality = 10
         Solution objective = 0.00230681
     Portfolio rMV variance = 0.00230681
      Sum of asset weights =  1.0000000

       Portfolio rMV return =   0.00338
    Portfolio minVar return =   0.00327
      Portfolio init return =   0.00382

      Portfolio rMV st.dev. =   0.04803
   Portfolio minVar st.dev. =   0.04358
     Portfolio init st.dev. =   0.05151

   Portfolio rMV r.est.err. =   0.01839
 Portfolio r.est.err. bound =   0.01839
Portfolio minVar r.est.err. =   0.02918
  Portfolio init r.est.err. =   0.01839

   # of zero portf. weights = 0
  # of positive pf. weights = 10


Round near-zero portfolio weights, if necessary.

In [31]:
import copy
# Round near-zero portfolio weights
w_rMV = np.array(w_rMV)
w_rMV_nonrnd = copy.deepcopy(w_rMV)
w_rMV[w_rMV<1e-6] = 0
w_rMV = w_rMV / np.sum(w_rMV)
w_list = [w_rMV_nonrnd, w_rMV] 
# using zip() to print list vertically
print('Portfolio weights before and after rounding of near-zero elements:')
for w1, w2 in zip(*w_list):
    print('                {0:0.7f}   {1:0.7f}'.format(w1, w2))

Portfolio weights before and after rounding of near-zero elements:
                0.1174720   0.1174720
                0.1777841   0.1777841
                0.1230810   0.1230810
                0.1060153   0.1060153
                0.0518653   0.0518653
                0.0425496   0.0425496
                0.0290092   0.0290092
                0.1690974   0.1690974
                0.0969539   0.0969539
                0.0861722   0.0861722


Compare portfolio weights obtained from CPLEX and CVXPY.

In [32]:
wrMV_list = [w_rMV, w_rob1]
# using zip() to print list vertically
print('Portfolio weights obtained from CPLEX and CVXPY:')
for w1, w2 in zip(*wrMV_list):
    print('                {0:0.5f}   {1:0.5f}'.format(w1, w2))
print('\nDifference of two portfolios in L2-norm =', np.linalg.norm(w_rMV-w_rob1, ord=2))

Portfolio weights obtained from CPLEX and CVXPY:
                0.11747   0.11753
                0.17778   0.17772
                0.12308   0.12318
                0.10602   0.10595
                0.05187   0.05176
                0.04255   0.04253
                0.02901   0.02907
                0.16910   0.16909
                0.09695   0.09696
                0.08617   0.08621

Difference of two portfolios in L2-norm = 0.00019428706289649343
